In [23]:
import numpy as np
import pandas as pd
from datetime import datetime

import networkx as nx
from networkx import algorithms
import multiprocessing

In [37]:
class Node():
    def __init__(self, inter,ID):
        '''
        para::
            inter: the intersection where the node is located
        '''
        self.inter = inter
        self.id = ID

class Path():
    def __init__(self, nodelist):
        self.nodeList = nodelist
        self.len = len(nodelist)
    
    def append(self, node):
        self.nodeList.append(node)
        self.len += 1
    
    def travelTimePath(self):
        t = 0
        for i in range(self.len-1):
            t += travelTime(self.nodeList[i], self.nodeList[i+1])
        return t
        
class Trip():
    def __init__(self, oneTripData):
        '''
        parameters::
            o_inter: the intersection of the origin of this trip
            d_inter: the intersection of the destination of this trip
            o_time: the expected departure time of the trip
            d_time: the expected arrival time of the trip
        '''
        self.node_o = Node(oneTripData['ointer'], oneTripData.name)
        self.node_d = Node(oneTripData['dinter'], oneTripData.name)
        self.ointer = self.node_o.inter
        self.dinter = self.node_d.inter
        self.oTime = oneTripData['otime']
        self.dTime = oneTripData['dtime']
        self.id = oneTripData.name

# This function calculate the distance between the origin and destination of two trips 
# def odDistance(trip_1, trip_2):
#     '''
#     Input:
#         trip_1, trip_2: two instances of class Trip
#     Output:
#         oDis: the distance of these two trips' origin
#         dDis: the distance of these two trips' destination
#     return:
#         a tuple (oDis, dDis)
#     '''
#     o_a = trip_1.oLong - trip_2.oLong
#     o_b = trip_1.oLati - trip_2.oLati
#     oDis = 2*np.arcsin(np.sin(o_b/2)**2 
#                        + np.cos(trip_1.oLati)*np.cos(trip_2.oLati)*np.sin(o_a/2)**2)*6378.137
#     d_a = trip_1.dLong - trip_2.dLong
#     d_b = trip_1.dLati - trip_2.dLati
#     dDis = 2*np.arcsin(np.sin(d_b/2)**2 
#                        + np.cos(trip_1.dLati)*np.cos(trip_2.dLati)*np.sin(d_a/2)**2)*6378.137
#     return (oDis, dDis)

    
def travelTime(node_1, node_2):
    '''
    parameters::
        distanceMatrixDict:
        node_1:
        node_2:
    return:
        
    '''
    return distanceMatrixDict[(node_1.inter, node_2.inter)]['time']

def isSharable(Trip_1, Trip_2, timePenalty = 600, distancePenalty = 1, coveragePenalty = 1):
    '''
    Input:
        trip_1, trip_2: two instances of class Trip
    Output:
        
    return:
        
    '''
    if (Trip_1.oTime > Trip_2.oTime):
        trip_1 = Trip_2
        trip_2 = Trip_1
    else:
        trip_1 = Trip_1
        trip_2 = Trip_2
    # case 1
    if (trip_1.dTime > trip_2.dTime):
        if(trip_1.oTime + travelTime(trip_1.node_o, trip_2.node_o) < trip_2.oTime):
            path1 = Path([trip_1.node_o, trip_2.node_o, trip_2.node_d, trip_1.node_d])
            if(trip_1.dTime-timePenalty < trip_1.oTime+path1.travelTimePath() < trip_1.dTime):
                path2 = Path([trip_1.node_o, trip_2.node_o, trip_2.node_d])
                if(trip_1.dTime-timePenalty < trip_1.oTime+path2.travelTimePath() < trip_1.dTime):
                    if(path1.travelTimePath() < distancePenalty*travelTime(trip_1.node_o, trip_1.node_d)):
                        return True
                    else:
                        return False
                else:
                    return False
            else:
                return False
        else:
            return False
    else:
        if (trip_2.oTime < trip_1.dTime < trip_2.dTime):
            if(trip_1.oTime + travelTime(trip_1.node_o, trip_2.node_o) < trip_2.oTime):
                path1 = Path([trip_1.node_o, trip_2.node_o, trip_1.node_d])
                if(trip_1.dTime-timePenalty < trip_1.oTime+path1.travelTimePath() < trip_1.dTime):
                    path2 = Path([trip_1.node_o, trip_2.node_o, trip_1.node_d, trip_2.node_d])
                    if(trip_2.dTime-timePenalty < trip_1.oTime+path2.travelTimePath() < trip_2.dTime):
                        if(path1.travelTimePath() < distancePenalty*travelTime(trip_1.node_o, trip_1.node_d)):
                            path3 = Path([trip_2.node_o, trip_1.node_d, trip_2.node_d])
                            if(path3.travelTimePath() < distancePenalty*travelTime(trip_2.node_o, trip_2.node_d)):
                                if(travelTime(trip_1.node_o, trip_2.node_o) < coveragePenalty*travelTime(trip_1.node_o, trip_1.node_d)):
                                    return True
                            else:
                                return False
                        else:
                            return False
                    else:
                        return False
                else:
                    return False
            else:
                return False
        else:
            return False
    return False

class TripEdge():
    def __init__(self, pre_node, suc_node, weight):
        self.node_1 = pre_node
        self.node_2 = suc_node
        self.w = weight
        self.edge = (self.node_1, self.node_2, self.w)
        
def addTrip(i, tripData):
    trip_list = [Trip(tripData.iloc[i])]
    return trip_list
    
def generateTripList(tripData):
    cores = multiprocessing.cpu_count()
    pool = multiprocessing.Pool(processes = cores-2)
    trip_list = []
    for trips in pool.starmap(addTrip, zip(range(len(tripData)), [tripData]*len(tripData))):
        trip_list += trips
    pool.close()
    return trip_list

def addEdge(i,trip_list):
    trip_edges = []
    for j in range(i+1, len(trip_list)):
        # print('Epoch: %d' % (i+j))
        if isSharable(trip_list[i], trip_list[j]):
            weight = 1
#             print('These two trips are sharable')
            trip_edges.append((trip_list[i].id,trip_list[j].id))
#             print('=='*10+'Trip Edge List:')
#             print(trip_edge_list)
#             if trip_list[i].oTime > trip_list[j].oTime:
#                 trip_edges.append((trip_list[i].node_d, trip_list[j].node_o))
#             else:
#                 trip_edges.append((trip_list[j].node_d, trip_list[i].node_o))
    return trip_edges

def generateTripEdgeList(trip_list):
    cores = multiprocessing.cpu_count()
    pool = multiprocessing.Pool(processes = cores-2)
    # global trip_edge_list
    trip_edge_list = []
    for trip_edge in pool.starmap(addEdge,zip(range(len(trip_list)-1),[trip_list]*(len(trip_list)-1))):
#         print('=='*10+'Trip Edge:')
#         print(trip_edge)
        trip_edge_list += trip_edge
#     cnt = 0
#     for _ in pool.starmap(addEdge,zip(range(len(trip_list)-1),[trip_list]*(len(trip_list)-1),[trip_edge_list]*(len(trip_list)-1))):
#         print('Done: %d/%d\r' % (cnt, len(trip_list)-1))
#         cnt += 1
    pool.close()
    return trip_edge_list
                
class TripGraph():
    def __init__(self, trip_edge_list):
        self.trip_edge_list = trip_edge_list
        self.num_trips = len(trip_edge_list)
        self.g = nx.Graph()
        self.g.add_edges_from(self.trip_edge_list,weight=1)
        print(self.g)
        
    def addTripEdge(self, trip_edge):
        self.trip_edge_list.append(trip_edge)
        self.num_trips += 1

    def maxMatching(self):
        graphs = list(nx.connected_component_subgraphs(self.g))
        self.matches = {}
        for graph in graphs:
            self.matches.update(algorithms.matching.max_weight_matching(graph))
        self.number_match = len(self.matches)
        return (self.number_match, self.matches)
    
    def minFleeting(self):
        return self.number_match

In [3]:
start = datetime.now()

matrixPath = r'./od_time.xlsx'
tripPath = r'./trip_list.xlsx'
tripData = pd.read_excel(tripPath)
tripData = tripData.set_index('trip_rowid')
distanceMatrix = pd.read_excel(matrixPath)
distanceMatrix = distanceMatrix.sort_values(by = 'time')
distanceMatrix = distanceMatrix.sort_values(by = 'o')
distanceMatrix = distanceMatrix.set_index(list(distanceMatrix.columns[:2]))
distanceMatrixDict = distanceMatrix.to_dict(orient='index')

In [28]:
start = datetime.now()
interval = 600
all_fleet = 0
matching = {}
# int(86400/interval)
for i in range(int(86400/interval)):
    tripdata = tripData[tripData['otime'] <= interval*(i+1)]
    tripdata = tripdata[interval*i <= tripdata['otime']]
    print('=='*20)
    pre = datetime.now()
    print('Data Preprocessing Finished in: %.2f s' % (pre-start).total_seconds())
    trip_list = generateTripList(tripdata)
    print('=='*20)
    tripTime = datetime.now()
    print('Trip List Generated in: %.2f s' % (tripTime-pre).total_seconds())
    trip_edge_list = generateTripEdgeList(trip_list)
    print('=='*20)
    edgeTime = datetime.now()
    print('Trip Edge List Generated in: %.2f s' % (edgeTime-tripTime).total_seconds())
    trip_graph = TripGraph(trip_edge_list)
    graphTime = datetime.now()
    print('=='*20)
    print('Trip Graph Initialized in: %.2f s' % (graphTime-edgeTime).total_seconds())
    (number_match, matches) = trip_graph.maxMatching()
    matching.update(matches)
    minFleeting = trip_graph.minFleeting()
    all_fleet += minFleeting
    print('=='*20)
    print('The Optimal Result Found is: %d' % minFleeting)
    end = datetime.now()
    print('=='*20)
    print('The Time Spent was: %.2f s' % (end-start).total_seconds())
end = datetime.now()
print('=='*20)
print('Total Time Spent: %.2f s'%(end-start).total_seconds())

Data Preprocessing Finished in: 0.00 s
Trip List Generated in: 0.19 s
Trip Edge List Generated in: 0.48 s

Trip Graph Initialized in: 0.00 s
The Optimal Result Found is: 2
The Time Spent was: 0.68 s
Data Preprocessing Finished in: 0.68 s
Trip List Generated in: 0.21 s
Trip Edge List Generated in: 0.35 s

Trip Graph Initialized in: 0.00 s
The Optimal Result Found is: 0
The Time Spent was: 1.25 s
Data Preprocessing Finished in: 1.25 s
Trip List Generated in: 0.20 s
Trip Edge List Generated in: 0.35 s

Trip Graph Initialized in: 0.00 s
The Optimal Result Found is: 0
The Time Spent was: 1.80 s
Data Preprocessing Finished in: 1.81 s
Trip List Generated in: 0.18 s
Trip Edge List Generated in: 0.31 s

Trip Graph Initialized in: 0.00 s
The Optimal Result Found is: 0
The Time Spent was: 2.30 s
Data Preprocessing Finished in: 2.31 s
Trip List Generated in: 0.18 s
Trip Edge List Generated in: 0.31 s

Trip Graph Initialized in: 0.00 s
The Optimal Result Found is: 0
The Time Spent was: 2.80 s
Data 

Trip List Generated in: 0.19 s
Trip Edge List Generated in: 0.25 s

Trip Graph Initialized in: 0.00 s
The Optimal Result Found is: 0
The Time Spent was: 9.49 s
Data Preprocessing Finished in: 9.50 s
Trip List Generated in: 0.18 s
Trip Edge List Generated in: 0.25 s

Trip Graph Initialized in: 0.00 s
The Optimal Result Found is: 0
The Time Spent was: 9.93 s
Data Preprocessing Finished in: 9.94 s
Trip List Generated in: 0.17 s
Trip Edge List Generated in: 0.27 s

Trip Graph Initialized in: 0.00 s
The Optimal Result Found is: 0
The Time Spent was: 10.38 s
Data Preprocessing Finished in: 10.39 s
Trip List Generated in: 0.17 s
Trip Edge List Generated in: 0.25 s

Trip Graph Initialized in: 0.00 s
The Optimal Result Found is: 0
The Time Spent was: 10.80 s
Data Preprocessing Finished in: 10.81 s
Trip List Generated in: 0.18 s
Trip Edge List Generated in: 0.26 s

Trip Graph Initialized in: 0.00 s
The Optimal Result Found is: 0
The Time Spent was: 11.25 s
Data Preprocessing Finished in: 11.25 s

Trip List Generated in: 0.20 s
Trip Edge List Generated in: 0.53 s

Trip Graph Initialized in: 0.00 s
The Optimal Result Found is: 1
The Time Spent was: 20.20 s
Data Preprocessing Finished in: 20.21 s
Trip List Generated in: 0.26 s
Trip Edge List Generated in: 1.00 s

Trip Graph Initialized in: 0.00 s
The Optimal Result Found is: 9
The Time Spent was: 21.47 s
Data Preprocessing Finished in: 21.48 s
Trip List Generated in: 0.21 s
Trip Edge List Generated in: 2.45 s

Trip Graph Initialized in: 0.00 s
The Optimal Result Found is: 39
The Time Spent was: 24.16 s
Data Preprocessing Finished in: 24.17 s
Trip List Generated in: 0.27 s
Trip Edge List Generated in: 3.07 s

Trip Graph Initialized in: 0.00 s
The Optimal Result Found is: 50
The Time Spent was: 27.54 s
Data Preprocessing Finished in: 27.54 s
Trip List Generated in: 0.33 s
Trip Edge List Generated in: 4.76 s

Trip Graph Initialized in: 0.00 s
The Optimal Result Found is: 97
The Time Spent was: 32.72 s
Data Preprocessing Finished in: 

Trip List Generated in: 0.29 s
Trip Edge List Generated in: 4.53 s

Trip Graph Initialized in: 0.00 s
The Optimal Result Found is: 106
The Time Spent was: 125.23 s
Data Preprocessing Finished in: 125.24 s
Trip List Generated in: 0.31 s
Trip Edge List Generated in: 4.14 s

Trip Graph Initialized in: 0.00 s
The Optimal Result Found is: 100
The Time Spent was: 129.85 s
Data Preprocessing Finished in: 129.86 s
Trip List Generated in: 0.22 s
Trip Edge List Generated in: 4.45 s

Trip Graph Initialized in: 0.00 s
The Optimal Result Found is: 114
The Time Spent was: 134.68 s
Data Preprocessing Finished in: 134.69 s
Trip List Generated in: 0.23 s
Trip Edge List Generated in: 4.14 s

Trip Graph Initialized in: 0.00 s
The Optimal Result Found is: 92
The Time Spent was: 139.15 s
Data Preprocessing Finished in: 139.15 s
Trip List Generated in: 0.24 s
Trip Edge List Generated in: 3.75 s

Trip Graph Initialized in: 0.00 s
The Optimal Result Found is: 89
The Time Spent was: 143.21 s
Data Preprocessing

Trip List Generated in: 0.23 s
Trip Edge List Generated in: 2.58 s

Trip Graph Initialized in: 0.00 s
The Optimal Result Found is: 38
The Time Spent was: 200.63 s
Data Preprocessing Finished in: 200.65 s
Trip List Generated in: 0.24 s
Trip Edge List Generated in: 2.90 s

Trip Graph Initialized in: 0.00 s
The Optimal Result Found is: 41
The Time Spent was: 203.81 s
Data Preprocessing Finished in: 203.82 s
Trip List Generated in: 0.29 s
Trip Edge List Generated in: 2.93 s

Trip Graph Initialized in: 0.00 s
The Optimal Result Found is: 46
The Time Spent was: 207.06 s
Data Preprocessing Finished in: 207.07 s
Trip List Generated in: 0.29 s
Trip Edge List Generated in: 3.26 s

Trip Graph Initialized in: 0.00 s
The Optimal Result Found is: 48
The Time Spent was: 210.64 s
Data Preprocessing Finished in: 210.65 s
Trip List Generated in: 0.24 s
Trip Edge List Generated in: 4.07 s

Trip Graph Initialized in: 0.00 s
The Optimal Result Found is: 77
The Time Spent was: 215.00 s
Data Preprocessing Fi

Trip List Generated in: 0.24 s
Trip Edge List Generated in: 4.60 s

Trip Graph Initialized in: 0.00 s
The Optimal Result Found is: 89
The Time Spent was: 286.88 s
Data Preprocessing Finished in: 286.89 s
Trip List Generated in: 0.24 s
Trip Edge List Generated in: 4.88 s

Trip Graph Initialized in: 0.00 s
The Optimal Result Found is: 119
The Time Spent was: 292.07 s
Data Preprocessing Finished in: 292.09 s
Trip List Generated in: 0.24 s
Trip Edge List Generated in: 4.35 s

Trip Graph Initialized in: 0.01 s
The Optimal Result Found is: 118
The Time Spent was: 296.86 s
Data Preprocessing Finished in: 296.87 s
Trip List Generated in: 0.24 s
Trip Edge List Generated in: 4.98 s

Trip Graph Initialized in: 0.01 s
The Optimal Result Found is: 127
The Time Spent was: 302.25 s
Data Preprocessing Finished in: 302.26 s
Trip List Generated in: 0.23 s
Trip Edge List Generated in: 4.71 s

Trip Graph Initialized in: 0.01 s
The Optimal Result Found is: 130
The Time Spent was: 307.40 s
Data Preprocessin

Trip List Generated in: 0.24 s
Trip Edge List Generated in: 2.56 s

Trip Graph Initialized in: 0.00 s
The Optimal Result Found is: 37
The Time Spent was: 377.28 s
Data Preprocessing Finished in: 377.30 s
Trip List Generated in: 0.25 s
Trip Edge List Generated in: 2.18 s

Trip Graph Initialized in: 0.00 s
The Optimal Result Found is: 34
The Time Spent was: 379.75 s
Data Preprocessing Finished in: 379.77 s
Trip List Generated in: 0.25 s
Trip Edge List Generated in: 2.13 s

Trip Graph Initialized in: 0.00 s
The Optimal Result Found is: 28
The Time Spent was: 382.16 s
Data Preprocessing Finished in: 382.18 s
Trip List Generated in: 0.22 s
Trip Edge List Generated in: 2.02 s

Trip Graph Initialized in: 0.00 s
The Optimal Result Found is: 32
The Time Spent was: 384.44 s
Data Preprocessing Finished in: 384.46 s
Trip List Generated in: 0.28 s
Trip Edge List Generated in: 2.09 s

Trip Graph Initialized in: 0.00 s
The Optimal Result Found is: 34
The Time Spent was: 386.84 s
Data Preprocessing Fi

Trip List Generated in: 0.21 s
Trip Edge List Generated in: 0.86 s

Trip Graph Initialized in: 0.00 s
The Optimal Result Found is: 7
The Time Spent was: 413.78 s
Data Preprocessing Finished in: 413.80 s
Trip List Generated in: 0.20 s
Trip Edge List Generated in: 0.81 s

Trip Graph Initialized in: 0.00 s
The Optimal Result Found is: 4
The Time Spent was: 414.81 s
Data Preprocessing Finished in: 414.84 s
Trip List Generated in: 0.18 s
Trip Edge List Generated in: 0.67 s

Trip Graph Initialized in: 0.00 s
The Optimal Result Found is: 4
The Time Spent was: 415.68 s
Data Preprocessing Finished in: 415.71 s
Trip List Generated in: 0.20 s
Trip Edge List Generated in: 0.60 s

Trip Graph Initialized in: 0.00 s
The Optimal Result Found is: 0
The Time Spent was: 416.50 s
Data Preprocessing Finished in: 416.52 s
Trip List Generated in: 0.20 s
Trip Edge List Generated in: 0.54 s

Trip Graph Initialized in: 0.00 s
The Optimal Result Found is: 2
The Time Spent was: 417.27 s
Data Preprocessing Finishe

In [26]:
all_fleet

9404

In [35]:
matching

{304456: 324642,
 164648: 292356,
 234528: 47534,
 314314: 47895,
 208831: 294351,
 80429: 54983,
 214968: 266225,
 226517: 243261,
 317635: 248684,
 327966: 23319,
 256392: 295596,
 72450: 122404,
 297448: 47142,
 225445: 213438,
 197954: 107255,
 258193: 318799,
 243528: 102681,
 234766: 75271,
 156233: 193506,
 210266: 215147,
 285213: 133910,
 263144: 958,
 262320: 157775,
 324986: 238884,
 324613: 27918,
 44414: 38943,
 138649: 47137,
 133737: 12516,
 318937: 30663,
 291331: 246269,
 242728: 298434,
 54120: 228550,
 67073: 79716,
 207971: 117639,
 178753: 117770,
 259288: 181628,
 268955: 207884,
 82004: 303198,
 212680: 226675,
 209470: 33278,
 74555: 156973,
 8692: 31359,
 269251: 9196,
 64362: 29403,
 39740: 64310,
 302978: 39389,
 217576: 35018,
 213321: 43269,
 88944: 44833,
 66445: 189726,
 131547: 206311,
 91417: 132255,
 165058: 81357,
 97520: 256500,
 197977: 105621,
 228585: 198935,
 161120: 144196,
 251704: 231372,
 312680: 277839,
 78821: 108918,
 185552: 198942,
 2577

In [36]:
tripData.head(5)

,otime,dtime,ointer,dinter
trip_rowid,,,,
1,19728,20024,36,31
2,31514,31781,72,64
3,54911,54947,31,72
4,24257,24307,60,67
5,33788,34390,20,69


In [40]:
distanceMatrix

time
o  d       
1  1     17
   2     55
   3    250
   4    422
   5    630
   6    770
   7    234
   8    795
   9    862
   10   970
   11  1139
   12   163
   13   613
   14  1064
   15   866
   16   981
   17  1392
   18   217
   19   374
   20   712
   21   767
   22   892
   23  1057
   24  1240
   25  1189
   26  1435
   27  1350
   28   198
   29   341
   30   874
...     ...
78 48  1200
   49  1464
   51  1091
   52  1406
   53   498
   54   808
   55   866
   56   698
   57   530
   58   649
   59   910
   60  2003
   61  1212
   62  1711
   63  1790
   64   378
   65   336
   66  1023
   67  1127
   68   624
   69   659
   70  1259
   71  1355
   72   628
   73   753
   74   108
   75  1732
   76  2021
   77   951
   78    10

[4489 rows x 1 columns]